In [1]:
import requests
import json

url="https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5"
res=requests.get(url)
d=json.loads(res.text)
data_all=json.loads(d["data"])
data_all.keys()

dict_keys(['lastUpdateTime', 'chinaTotal', 'chinaAdd', 'isShowAdd', 'showAddSwitch', 'areaTree', 'chinaDayList', 'chinaDayAddList', 'dailyNewAddHistory', 'dailyHistory', 'wuhanDayList', 'articleList'])

In [2]:
url2="https://view.inews.qq.com/g2/getOnsInfo?name=disease_other"
res2=requests.get(url2)
d2=json.loads(res2.text)
data_other=json.loads(d2["data"])
data_other.keys()

dict_keys(['chinaDayList', 'chinaDayAddList', 'dailyNewAddHistory', 'dailyHistory', 'wuhanDayList', 'articleList', 'provinceCompare', 'foreignList', 'globalStatis', 'globalDailyHistory', 'cityStatis'])

In [24]:
print(data_other["dailyNewAddHistory"])

[{'date': '01.20', 'hubei': 72, 'country': 77, 'notHubei': 5}, {'date': '01.21', 'hubei': 105, 'country': 149, 'notHubei': 44}, {'date': '01.22', 'hubei': 69, 'country': 131, 'notHubei': 62}, {'date': '01.23', 'hubei': 105, 'country': 259, 'notHubei': 154}, {'date': '01.24', 'hubei': 180, 'country': 444, 'notHubei': 264}, {'date': '01.25', 'hubei': 323, 'country': 688, 'notHubei': 365}, {'date': '01.26', 'hubei': 371, 'country': 769, 'notHubei': 398}, {'date': '01.27', 'hubei': 1291, 'country': 1771, 'notHubei': 480}, {'date': '01.28', 'hubei': 840, 'country': 1459, 'notHubei': 619}, {'date': '01.29', 'hubei': 1032, 'country': 1737, 'notHubei': 705}, {'date': '01.30', 'hubei': 1220, 'country': 1982, 'notHubei': 762}, {'date': '01.31', 'hubei': 1347, 'country': 2102, 'notHubei': 755}, {'date': '02.01', 'hubei': 1921, 'country': 2590, 'notHubei': 669}, {'date': '02.02', 'hubei': 2103, 'country': 2829, 'notHubei': 726}, {'date': '02.03', 'hubei': 2345, 'country': 3235, 'notHubei': 890}, {

In [3]:
import pymysql
import time
import json
import traceback


In [4]:
def get_tencent_data():  # 获取腾讯历史数据和当日详细数据
    url1 = "https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5"
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'
    }
    r1 = requests.get(url1, headers)
    res1 = json.loads(r1.text)  # json转换字符串
    data_all = json.loads(res1['data'])
    # 获取['lastUpdateTime', 'chinaTotal', 'chinaAdd', 'isShowAdd', 'showAddSwitch', 'areaTree',
    # 'chinaDayList', 'chinaDayAddList', 'dailyNewAddHistory', 'dailyHistory', 'wuhanDayList', 'articleList']

    url2 = "https://view.inews.qq.com/g2/getOnsInfo?name=disease_other"
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'
    }
    r2 = requests.get(url2, headers)
    res2 = json.loads(r2.text)  # json转换字符串
    data_other = json.loads(res2['data'])
    # 获取['chinaDayList', 'chinaDayAddList', 'dailyNewAddHistory', 'dailyHistory', 'wuhanDayList', 'articleList',
    # 'provinceCompare', 'foreignList', 'globalStatis', 'globalDailyHistory']

    hisrory = {}  # 历史数据
    for i in data_other["chinaDayList"]:
        ds = "2020." + i["date"]
        tup = time.strptime(ds, "%Y.%m.%d")
        ds = time.strftime("%Y-%m-%d", tup)  # 改变时间格式，数据库是datatime格式
        confirm = i["confirm"]
        suspect = i["suspect"]
        heal = i["heal"]
        dead = i["dead"]
        hisrory[ds] = {"confirm": confirm, "suspect": suspect, "heal": heal, "dead": dead}

    for i in data_other["chinaDayAddList"]:
        ds = "2020." + i["date"]
        tup = time.strptime(ds, "%Y.%m.%d")
        ds = time.strftime("%Y-%m-%d", tup)  # 改变时间格式，数据库是datatime格式
        confirm = i["confirm"]
        suspect = i["suspect"]
        heal = i["heal"]
        dead = i["dead"]
        hisrory[ds].update({"confirm": confirm, "suspect": suspect, "heal": heal, "dead": dead})

    details=[] #当日详细数据
    update_time=data_all["lastUpdateTime"]
    data_country=data_all["areaTree"]#list25个国家
    data_province=data_country[0]["children"]#中国各省
    for pro_infos in data_province:
        province=pro_infos["name"]
        for city_infos in pro_infos["children"]:
            city=city_infos["name"]
            confirm=city_infos["total"]["confirm"]
            confirm_add=city_infos["today"]["confirm"]
            heal=city_infos["total"]["heal"]
            dead=city_infos["total"]["dead"]
            details.append([update_time,province,city,confirm,confirm_add,heal,dead])

    return hisrory,details

In [ ]:
存取腾讯数据

In [11]:
#pymysql简单使用
#建立连接
conn=pymysql.connect(host="localhost",
                    user="root",
                    password="123456",
                    db="cov")
#创建游标，默认是元祖型
cursor=conn.cursor()

sql="select * from history"
cursor.execute(sql)
res=cursor.fetchall()
print(res)

cursor.close()
conn.close()

((datetime.datetime(2020, 1, 13, 0, 0), 41, None, 0, None, 0, None, 1, None), (datetime.datetime(2020, 1, 14, 0, 0), 41, None, 0, None, 0, None, 1, None), (datetime.datetime(2020, 1, 15, 0, 0), 41, None, 0, None, 5, None, 2, None), (datetime.datetime(2020, 1, 16, 0, 0), 45, None, 0, None, 8, None, 2, None), (datetime.datetime(2020, 1, 17, 0, 0), 62, None, 0, None, 12, None, 2, None), (datetime.datetime(2020, 1, 18, 0, 0), 198, None, 0, None, 17, None, 3, None), (datetime.datetime(2020, 1, 19, 0, 0), 275, None, 0, None, 18, None, 4, None), (datetime.datetime(2020, 1, 20, 0, 0), 77, None, 27, None, 0, None, 0, None), (datetime.datetime(2020, 1, 21, 0, 0), 149, None, 53, None, 0, None, 3, None), (datetime.datetime(2020, 1, 22, 0, 0), 131, None, 257, None, 0, None, 8, None), (datetime.datetime(2020, 1, 23, 0, 0), 259, None, 680, None, 6, None, 8, None), (datetime.datetime(2020, 1, 24, 0, 0), 444, None, 1118, None, 3, None, 16, None), (datetime.datetime(2020, 1, 25, 0, 0), 688, None, 1309, 

In [5]:
#pymysql简单使用
#建立连接
conn=pymysql.connect(host="localhost",
                    user="root",
                    password="123456",
                    db="cov")
#创建游标，默认是元祖型
cursor=conn.cursor()

sql="insert into history values(%s,%s,%s,%s,%s,%s,%s,%s,%s)"
cursor.execute(sql,[time.strftime("%Y-%m-%d"),10,1,2,3,4,5,6,7])
conn.commit()#提交事务

cursor.close()
conn.close()

In [6]:
def get_conn():# 封装创建连接
    """
    :return: 连接，游标
    """
    
    conn = pymysql.connect(host="localhost",
                           user="root",
                           password="123456",
                           db="cov",
                           charset="utf8")
    # 创建游标
    cursor = conn.cursor()# 执行完毕返回的结果集默认以元组显示
    return conn, cursor

def close_conn(conn, cursor):
    if cursor:
        cursor.close()
    if conn:
        conn.close()

In [7]:
def update_details():
    """
    更新 details 表
    :return:
    """
    cursor = None
    conn = None
    try:
        li = get_tencent_data()[1]  #  0 是历史数据字典,1 最新详细数据列表
        conn, cursor = get_conn()
        sql = "insert into details(update_time,province,city,confirm,confirm_add,heal,dead) values(%s,%s,%s,%s,%s,%s,%s)"
        sql_query = 'select %s=(select update_time from details order by id desc limit 1)' #对比当前最大时间戳
        cursor.execute(sql_query,li[0][0])
        if not cursor.fetchone()[0]:#和第一条数据相比判断是否更新
            print(f"{time.asctime()}开始更新最新数据")
            for item in li:
                cursor.execute(sql, item)
            conn.commit()  # 提交事务 update delete insert操作
            print(f"{time.asctime()}更新最新数据完毕")
        else:
            print(f"{time.asctime()}已是最新数据！")
    except:
        traceback.print_exc()
    finally:
        close_conn(conn, cursor)

In [8]:
def insert_history():
    """
        插入历史数据
    :return:
    """
    cursor = None
    conn = None
    try:
        dic = get_tencent_data()[0]  # 0 是历史数据字典,1 最新详细数据列表
        print(f"{time.asctime()}开始插入历史数据")
        conn, cursor = get_conn()
        sql = "insert into history values(%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        for k, v in dic.items():
            # item 格式 {'2020-01-13': {'confirm': 41, 'suspect': 0, 'heal': 0, 'dead': 1}
            cursor.execute(sql, [k, v.get("confirm"), v.get("confirm_add"), v.get("suspect"),
                                 v.get("suspect_add"), v.get("heal"), v.get("heal_add"),
                                 v.get("dead"), v.get("dead_add")])

        conn.commit()  # 提交事务 update delete insert操作
        print(f"{time.asctime()}插入历史数据完毕")
    except:
        traceback.print_exc()
    finally:
        close_conn(conn, cursor)

In [9]:
def update_history():
    """
    更新历史数据
    :return:
    """
    cursor = None
    conn = None
    try:
        dic = get_tencent_data()[0]  #  0 是历史数据字典,1 最新详细数据列表
        print(f"{time.asctime()}开始更新历史数据")
        conn, cursor = get_conn()
        sql = "insert into history values(%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        sql_query = "select confirm from history where ds=%s"
        for k, v in dic.items():
            # item 格式 {'2020-01-13': {'confirm': 41, 'suspect': 0, 'heal': 0, 'dead': 1}
            if not cursor.execute(sql_query, k):
                cursor.execute(sql, [k, v.get("confirm"), v.get("confirm_add"), v.get("suspect"),
                                     v.get("suspect_add"), v.get("heal"), v.get("heal_add"),
                                     v.get("dead"), v.get("dead_add")])
        conn.commit()  # 提交事务 update delete insert操作
        print(f"{time.asctime()}历史数据更新完毕")
    except:
        traceback.print_exc()
    finally:
        close_conn(conn, cursor)

In [9]:
insert_history()

Tue Mar 10 13:29:31 2020开始插入历史数据


Traceback (most recent call last):
  File "<ipython-input-7-aa661a7cfd88>", line 17, in insert_history
    v.get("dead"), v.get("dead_add")])
  File "d:\python\lib\site-packages\pymysql\cursors.py", line 170, in execute
    result = self._query(query)
  File "d:\python\lib\site-packages\pymysql\cursors.py", line 328, in _query
    conn.query(q)
  File "d:\python\lib\site-packages\pymysql\connections.py", line 517, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "d:\python\lib\site-packages\pymysql\connections.py", line 732, in _read_query_result
    result.read()
  File "d:\python\lib\site-packages\pymysql\connections.py", line 1075, in read
    first_packet = self.connection._read_packet()
  File "d:\python\lib\site-packages\pymysql\connections.py", line 684, in _read_packet
    packet.check_error()
  File "d:\python\lib\site-packages\pymysql\protocol.py", line 220, in check_error
    err.raise_mysql_exception(self._data)
  File "d:\python\lib\

In [10]:
update_history()

Wed Mar 11 13:06:39 2020开始更新历史数据
Wed Mar 11 13:06:39 2020历史数据更新完毕


In [11]:
update_details()

Wed Mar 11 13:06:43 2020开始更新最新数据
Wed Mar 11 13:06:44 2020更新最新数据完毕


In [ ]:
##爬取百度热搜

In [12]:
from selenium.webdriver import Chrome,ChromeOptions

option=ChromeOptions()
option.add_argument("--headless")#无头模式，隐藏浏览器
option.add_argument("--no-sandbox")

browser=Chrome(options=option)
url="https://voice.baidu.com/act/virussearch/virussearch?from=osari_map&tab=0&infomore=1"
browser.get(url)
but=browser.find_element_by_css_selector('#ptab-0 > div > div.VirusHot_1-5-4_32AY4F.VirusHot_1-5-4_2RnRvg > section > div')
but.click()#点击展开
time.sleep(1)#等待1s

c=browser.find_elements_by_xpath('//*[@id="ptab-0"]/div/div[2]/section/a/div/span[2]')
for i in c:
    print(i.text)
    
browser.close()

新型肺炎实时动态1103940
日本禁止倒卖口罩6581268
意大利确诊病例数破万5121558
安徽一马拉松禁止湖北籍报名4299345
意大利式疫情小喇叭开播3964573
6人回国确诊 入境未如实申报3459845
泉州酒店搜救犬贝贝四肢感染3404986
3000多名小学生集体模仿名画2789942
习近平考察火神山医院2703506
英国卫生部长确诊新冠肺炎2504237
住华南海鲜市场43天未感染2249735
消防员泉州坍塌酒店现场痛哭2082808
意甲球员进球后呼吁待在家中1918989
韩国首尔办公楼27人集体感染1396172
北京供暖延长至3月22日1218903
泉州被救男孩长大后想当医生1030615
38岁民警抗疫40天殉职1027980
印尼人因新冠肺炎大量抢购生姜1019403
支援湖北护士给儿子取名李援鄂907986
泉州坍塌酒店发现大量现金824949


In [ ]:
##存放百度数据

In [13]:
def get_baidu_hot():
    """
    :return: 返回百度疫情热搜
    """
    option = ChromeOptions()  # 创建谷歌浏览器实例
    option.add_argument("--headless")  # 隐藏浏览器
    option.add_argument('--no-sandbox')

    url = "https://voice.baidu.com/act/virussearch/virussearch?from=osari_map&tab=0&infomore=1"
    browser = Chrome(options=option,executable_path="./chromedriver.exe")
    browser.get(url)
    # 找到展开按钮
    dl = browser.find_element_by_xpath('//*[@id="ptab-0"]/div/div[2]/section/div')
    dl.click()
    time.sleep(1)
    # 找到热搜标签
    c = browser.find_elements_by_xpath('//*[@id="ptab-0"]/div/div[2]/section/a/div/span[2]')
    context = [i.text for i in c]  # 获取标签内容
    print(context)
    return context

In [14]:
def update_hotsearch():
    """
    将疫情热搜插入数据库
    :return:
    """
    cursor = None
    conn = None
    try:
        context = get_baidu_hot()
        print(f"{time.asctime()}开始更新热搜数据")
        conn, cursor = get_conn()
        sql = "insert into hotsearch(dt,content) values(%s,%s)"
        ts = time.strftime("%Y-%m-%d %X")
        for i in context:
            cursor.execute(sql, (ts, i))  # 插入数据
        conn.commit()  # 提交事务保存数据
        print(f"{time.asctime()}数据更新完毕")
    except:
        traceback.print_exc()
    finally:
        close_conn(conn, cursor)

In [15]:
update_hotsearch()

['新型肺炎实时动态1103940', '日本禁止倒卖口罩6581268', '意大利确诊病例数破万5121558', '安徽一马拉松禁止湖北籍报名4299345', '意大利式疫情小喇叭开播3964573', '6人回国确诊 入境未如实申报3459845', '泉州酒店搜救犬贝贝四肢感染3404986', '3000多名小学生集体模仿名画2789942', '习近平考察火神山医院2703506', '英国卫生部长确诊新冠肺炎2504237', '住华南海鲜市场43天未感染2249735', '消防员泉州坍塌酒店现场痛哭2082808', '意甲球员进球后呼吁待在家中1918989', '韩国首尔办公楼27人集体感染1396172', '北京供暖延长至3月22日1218903', '泉州被救男孩长大后想当医生1030615', '38岁民警抗疫40天殉职1027980', '印尼人因新冠肺炎大量抢购生姜1019403', '支援湖北护士给儿子取名李援鄂907986', '泉州坍塌酒店发现大量现金824949']
Wed Mar 11 13:07:12 2020开始更新热搜数据
Wed Mar 11 13:07:12 2020数据更新完毕
